# Graphing Birth-Death-Dispersal Processes

Run each box in order for code to work



In [ ]:
## IMPORTS
import matplotlib.pyplot as plt
import random
import math as math
import networkx as nx
import numpy as np
import time

In [ ]:
## Gets a random number between 1 and 10 of nodes to die. Chooses a node to be 
## deleted (death process) based on the survival rate of each node. Once the 
## node to be removed is chosen it is removed from the graph and its fecundity
## and survival rates are removed from the two lists.
def death(G): ## Keeps track of survival list throughout instead of remaking it each time
    numdeath = int(np.random.uniform(10)) #arbitrary for now and gets the number of nodes to be 'killed'

    # This loop goes through each chosen index in the source list and removes them from the graph
    for i in range(numdeath):
        # This will choose a random index of a node in G weighted by survival rates
        source = random.choices(list(survival_rates.keys()), weights = list(survival_rates.values()))[0]
        G.remove_node(source) # remove from graph
        del survival_rates[source] # delete from the survival dictionary
        del fecundity_rates[source] # delete from the fecundity dictionary

    # After the removals have occured the graph's fecundity and survival need to be adjusted
    fecundity(G)
    survival_rate(G)


In [ ]:
## Function picks one node to be the source of a dispersal based on the fecundity
## of the node. Once the source node is identified the number of other nodes the
## source will disperse calculated as a function of the source degree. Then the
## children nodes are disperesed randomly within a radius defined by r_dispersal.
def disperse(G, r_dispersal, pos_n):
    # Chooses random node index in G weighted by fecundity    
    source = random.choices(list(fecundity_rates.keys()), weights = list(fecundity_rates.values()))[0] # random number of population based on fecundity 
    
    xicoord = G.nodes[source]["attr_dict"]['x'] # grab x
    yicoord = G.nodes[source]["attr_dict"]['y'] # grab y coordinate
        
    children = int(10 * math.exp(-0.1 * G.degree(source))) # number of children it will have, based on desmos
    for j in range(children):
        added = False
        while(added==False):
            # random radius length normally distributed and random angle uniformly distributed
            rand_r = np.random.uniform(r_dispersal) #radius between 0 and r_dispersal
            rand_theta = random.uniform(0,2*math.pi) #angle in radians
            xjcoord = rand_r*math.cos(rand_theta) + xicoord
            yjcoord = rand_r*math.sin(rand_theta) + yicoord
    
            # add the node
            if (xjcoord <= 2000 and yjcoord <= 2000 and xjcoord >= 0 and yjcoord >= 0): # Include to force window to be constant
                G.add_node(pos_n,attr_dict={"x":xjcoord,"y":yjcoord})
                connect(G, pos_n)
                pos_n = pos_n + 1
                added = True
    return pos_n

In [ ]:
## Function that takes the entire graph G, and calculates the distance between 
## all the nodes in the graph. When a the distance between two nodes in the 
## graph is found to be less than the radius 'r_nieghborhood' the two nodes are
## connected. At the end of the function the fecundity and survival rates are 
## recalculated.
def connect_all(G,r_nieghborhood):
    for i in G:
        xicoord = G.nodes[i]["attr_dict"]['x']
        yicoord = G.nodes[i]["attr_dict"]['y']
        for j in G:
            xjcoord = G.nodes[j]["attr_dict"]['x']
            yjcoord = G.nodes[j]["attr_dict"]['y']
            if (i!=j): # I do not need to connect a point to itself
                ridistance = math.sqrt((xicoord-xjcoord)**2+(yicoord-yjcoord)**2) # finding radial distance
                # if the distance between points indexed at i and j are less than my radius I append them to groupi
                if (ridistance <= r_nieghborhood):
                    #print(ridistance)
                    G.add_edge(i,j)
                    
    fecundity(G)
    survival_rate(G)

In [ ]:
## Function that takes a node 'node' and the entire graph G, and calculates the 
## distance between 'node' and all other nodes in the graph. When a the distance
## between 'node' and another node in the graph is found to be less than the 
## radius 'r_nieghborhood' the two nodes are connected. At the end of the 
## function the fecundity and survival rates are recalculated.
def connect(G, node):
    xicoord = G.nodes[node]["attr_dict"]['x']
    yicoord = G.nodes[node]["attr_dict"]['y']
    
    for j in G:
        xjcoord = G.nodes[j]["attr_dict"]['x']  #WM: I'm curious about why single quotes are used here as opposed to the double, used above
        yjcoord = G.nodes[j]["attr_dict"]['y']
        if (node!=j): # I do not need to connect a point to itself
            ridistance = math.sqrt((xicoord-xjcoord)**2+(yicoord-yjcoord)**2) # finding radial distance
            # if the distance between points indexed at i and j are less than my radius I append them to groupi
            if (ridistance <= r_nieghborhood):
                #print(ridistance)
                G.add_edge(node,j)
    
    fecundity(G)
    survival_rate(G)

In [ ]:
## Function will take our graph G and graph all nodes and their connections
def my_grapher(G):
    pos = nx.spring_layout(G) #WM: You may not need "spring_layout" here
    for i in G:
        pos[i]=[G.nodes[i]["attr_dict"]['x'],G.nodes[i]["attr_dict"]['y']]
   
    fig = plt.figure(facecolor='w')
    
    xmax = 2000
    xmin = 0
    ymax = 2000
    ymin = 0
    
    ax = fig.add_subplot(121)
    nx.draw(G,pos,node_size = 5) #to include ax again do ax=ax
    limits=plt.axis('on')
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

    plt.subplot(1, 2, 2)
    plt.bar(height=nx.degree_histogram(G),x=list(range(0,len(nx.degree_histogram(G)))))


In [ ]:
## This function applies a fecundity to each node based upon degree
## (higher degree means lower fecundity). The fecundity is used as weights
## to determine which node will be the source of a dispersal.
def fecundity(G):
    for i in G:
        fecundity_rates[i] = math.exp(-G.degree(i)) # to add in a fecundity dictionary

In [ ]:
## This function applies a survival rate to each node based upon degree (higher 
## degree means lower survival rate). The survival rate is used as weights
## to determine which node will be deleted from the graph (death process).
def survival_rate(G):
    for i in G:
        survival_rates[i] = math.exp(G.degree(i)) # Putting the survival rate for i as a function of the degrees of the node i

In [ ]:
## This function allows you to pick a specific windowed view and will show side
## by side a windowed view of the graph and a histograph that shows degree
## frquency within the window.
def window(G,xmin,xmax,ymin,ymax):
    window_degree = []
    for i in G:
        x = G.nodes[i]["attr_dict"]['x']
        y = G.nodes[i]["attr_dict"]['y']
        if (x>=xmin and x<=xmax and y>=ymin and y<=ymax):
            window_degree.append(G.degree[i])
            
    # GRAPH THE WINDOW
    pos = nx.spring_layout(G)
    for i in G:
        pos[i]=[G.nodes[i]["attr_dict"]['x'],G.nodes[i]["attr_dict"]['y']]
   
    fig = plt.figure(facecolor='w')
    
    ax = fig.add_subplot(121, xlim=(xmin,xmax), ylim=(ymin,ymax))
    nx.draw(G,pos,node_size = 5,ax=ax)
    limits=plt.axis('on')
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

    plt.subplot(1, 2, 2)
    plt.hist(window_degree)
    

# Start here to generate new data

In [ ]:
n = 500 # Initial number of points
y1 = 0 # Minimum y value
y2 = 2000 # Maximum y value
x1 = 0 # Minimum x value
x2 = 2000 # Maximum x value
pos_n = 0 # Used purely for indexing in the data structure that holds the nodes
r_dispersal = 100 # dispersal radius (set to 100)
r_nieghborhood = 100 # This is the radius used to connect points (set to 100)
G = nx.Graph() # Creating an empty graph
survival_rates = {} # this is a dictionary that holds the survival rate for each node in the graph indexed at the key
fecundity_rates = {} # empty dictionary that holds the fecundity as values and the node as key.

plt.rcParams['figure.figsize'] = [20, 8] # Configures window size. 
plt.rcParams['figure.dpi'] = 100 

# Instantiate the points to be plotted
for i in range(n):
    G.add_node(i,attr_dict={"x":random.randint(x1,x2),"y":random.randint(y1,y2)})
    pos_n = pos_n + 1

# Connect and graph initial random points.
connect_all(G,r_nieghborhood)
fecundity(G)
survival_rate(G)
my_grapher(G)

## Keep track of size of graph
Size = [n]

In [ ]:
# First 100 iterations
for i in range(100):
    pos_n = disperse(G,r_dispersal,pos_n)
    death(G)
    Size.append(G.size())

In [ ]:
print(G.size())
my_grapher(G)

In [ ]:
# First 1,000 iterations.
for i in range(900):
    pos_n = disperse(G,r_dispersal,pos_n)
    death(G)
    Size.append(G.size())

In [ ]:
print(G.size())
my_grapher(G)

In [ ]:
# First 10,000 iterations
for i in range(9000):
    pos_n = disperse(G,r_dispersal,pos_n)
    death(G)
    Size.append(G.size())

In [ ]:
print(G.size())
my_grapher(G)

In [ ]:
# 100,000 iterations reached
for i in range(90000):
    pos_n = disperse(G,r_dispersal,pos_n)
    death(G)
    Size.append(G.size())

In [ ]:
print(G.size())
my_grapher(G)

In [ ]:
# Plot the population size
plt.plot(range(len(Size)),Size)
plt.xlabel('Iterations')
plt.ylabel('Population Size');